In [1]:
import pandas as pd
import MeCab
from collections import Counter, defaultdict

In [2]:
tagger = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd -u /Users/yuki/foo/bar/user_dic2.dic")
tagger.parse("")

'EOS\n'

In [3]:
stop_words = ["♦","▪","⚪","・","◾",]

In [4]:
store_file = ["../pretest_kiyoda_dinner.csv",
              "../pretest_aozora_dinner.csv",
             "../pretest_mitani_lunch.csv",
             "../pretest_mitani_dinner.csv"]

In [5]:
lunch_file = ["../pretest_sushitsuu_lunch.csv",
              "../pretest_sushiichi_lunch.csv",
              "../pretest_sawada_lunch.csv",
              "../pretest_matsukan_lunch.csv",
              "../pretest_kiyoda_lunch.csv",
              "../pretest_imamura_lunch.csv",
              "../pretest_hatsune_lunch.csv",
              "../pretest_mitani_lunch.csv"
              ]
dinner_file = ["../pretest_sushitsuu_dinner.csv",
              "../pretest_sushiichi_dinner.csv",
              "../pretest_sawada_dinner.csv",
              "../pretest_matsukan_dinner.csv",
              "../pretest_aozora_dinner.csv",
              "../pretest_imamura_dinner.csv",
              "../pretest_hatsune_dinner.csv",
               "../pretest_mitani_dinner.csv"
              ]

In [6]:
akazu = ["赤酢"]
nigiri = ["握り","にぎり","ニギリ"]
shari = ["シャリ","コメ","米","ライス","こめ","しゃり"]
maguro = ["まぐろ","マグロ","鮪","赤身","トロ","とろ"]
wine = ["ワイン"]

In [7]:
akazu_ad = [["強い","強め","きつい","つよい","つよめ","効く"],
        ["あっさり","シンプル"],
        ["酸っぱい","すっぱい","酸味"]]

nigiri_ad = [["大きい","でかい","大きめ","ビッグ"],
            ["小さい","小ぶり","小さめ"],
            ["創作","奇想天外","工夫","独特","意外","面白い","おもしろい"]]

shari_ad = [["大きい","でかい","大きめ","ビッグ"],
           ["小さい","小ぶり","小さめ"],
           ["パラパラ","少ない"],
           ["塩気","塩","しょっぱい"],
           ["甘い","甘め","あまい","あまめ","砂糖"],
           ["酢","米酢","白酢"]]

maguro_ad = [["うまい","美味","美味しい","おいしい","良い","よい","いい"],
            ["臭い","くさい","生臭い"],
            ["あっさり"],
            ["こってり"],
            ["とろける","消える"],
            ["酸味"],
            ["熟成"]]

wine_ad = [["多い","豊富","品揃え","品ぞろえ","取り揃え"],
            ["相性","マリアージュ", "合う"]]

In [8]:
features = [akazu, nigiri, shari, maguro, wine]
features_ad = [akazu_ad, nigiri_ad, shari_ad, maguro_ad, wine_ad]

In [9]:
def tokenize_ja(text):
    node = tagger.parseToNode(str(text)) #分かち書きしたのが入っている。イテレータ
    while node:
        surface = node.surface.lower() #surfaceは単語の見た目　例：トろ
        feature = node.feature.split(",")
        hinshi = feature[0]
        genkei = feature[6]
        if surface not in stop_words and surface != "" and hinshi in ["形容詞", "動詞", "副詞", "助動詞", "接続詞", "連体詞", "感動詞", "接頭詞"]:
            yield genkei
        elif surface not in stop_words and surface != "" and hinshi == "名詞":
            yield surface
        node = node.next

In [10]:
def tokenize(content):
    return [token for token in tokenize_ja(content)]

In [11]:
"""
all_review_word_lunch = defaultdict(int)
all_review_word_dinner  =defaultdict(int)
num_word_lunch = 0
num_word_dinner = 0
all_dinner_pare = defaultdict(int)
all_lunch_pare = defaultdict(int)
for file in lunch_file:
    df=pd.read_csv(file)
    for review in df["lunch_review"]:
        li=tokenize(review)
        for l in li:
            all_review_word_lunch[l] += 1 #その単語の登場回数を1加える
            num_word_lunch += 1
for file in dinner_file:
    df=pd.read_csv(file)
    for review in df["dinner_review"]:
        li=tokenize(review)
        for l in li:
            all_review_word_dinner[l] += 1 #その単語の登場回数を1加える
            num_word_dinner += 1
"""

'\nall_review_word_lunch = defaultdict(int)\nall_review_word_dinner  =defaultdict(int)\nnum_word_lunch = 0\nnum_word_dinner = 0\nall_dinner_pare = defaultdict(int)\nall_lunch_pare = defaultdict(int)\nfor file in lunch_file:\n    df=pd.read_csv(file)\n    for review in df["lunch_review"]:\n        li=tokenize(review)\n        for l in li:\n            all_review_word_lunch[l] += 1 #その単語の登場回数を1加える\n            num_word_lunch += 1\nfor file in dinner_file:\n    df=pd.read_csv(file)\n    for review in df["dinner_review"]:\n        li=tokenize(review)\n        for l in li:\n            all_review_word_dinner[l] += 1 #その単語の登場回数を1加える\n            num_word_dinner += 1\n'

In [12]:
all_review_word_lunch = defaultdict(int)
all_review_word_dinner  =defaultdict(int)
num_word_lunch = 0
num_word_dinner = 0
all_lunch_pare = defaultdict(int)
all_dinner_pare = defaultdict(int)
for file in lunch_file:
    df=pd.read_csv(file)
    for review in df["lunch_review"]:
        for sentence in review.split("。"):
            t=tokenize(sentence)
            for tt in t:
                    num_word_lunch += 1
            for i,feature in enumerate(features):
                for tt in t:
                    if tt in feature:
                        all_review_word_lunch[feature[0]] += 1
                if len(list(set(feature) & set(t))) != 0:
                    for feature_ad in features_ad[i]:
                        li = list(set(feature_ad) & set(t))
                        if len(li) != 0:
                            for ii in li:
                                all_lunch_pare[(feature[0],feature_ad[0])] += 1

for file in dinner_file:
    df=pd.read_csv(file)
    for review in df["dinner_review"]:
        for sentence in review.split("。"):
            t=tokenize(sentence)
            for tt in t:
                    num_word_dinner += 1
                    
            for i,feature in enumerate(features):
                for tt in t:
                    if tt in feature:
                        all_review_word_dinner[feature[0]] += 1
                if len(list(set(feature) & set(t))) != 0:
                    for feature_ad in features_ad[i]:
                        li = list(set(feature_ad) & set(t))
                        if len(li) != 0:
                            for ii in li:
                                all_dinner_pare[(feature[0],feature_ad[0])] += 1

In [13]:
all_review_word_lunch

defaultdict(int, {'シャリ': 496, 'まぐろ': 855, '握り': 689, '赤酢': 17, 'ワイン': 96})

In [14]:
df = pd.read_csv(store_file[0])
pare = defaultdict(int)
n = 0
dic = defaultdict(int)
for review in df["dinner_review"]:
    sentences = review.split("。")
    #一つのレビューを文単位に分割
    for s in sentences:
        t = tokenize(s)
        n += len(t)
        for i,feature in enumerate(features):
            for tt in t:
                if tt in feature:
                    dic[feature[0]] += 1
            if len(list(set(feature) & set(t))) != 0:
                for feature_ad in features_ad[i]:
                    li = list(set(feature_ad) & set(t))
                    if len(li) != 0:
                        for ii in li:
                            pare[(feature[0],feature_ad[0])] += 1

In [15]:
dic2 = defaultdict(int)
word_times = 0 #全単語数
for review in df["dinner_review"]:
    token_list = tokenize(review) #各レビューを形態素解析
    for token in token_list:
        word_times += 1
        if (token in akazu) or (token in nigiri) or \
        (token in shari) or (token in maguro):
            dic2[token] += 1

In [16]:
l = []
for i, feature in enumerate(features):
    ll = [feature[0]+ " " + str(dic[feature[0]]) + "回"]
    for feature_ad in features_ad[i]:
        ll.append(feature_ad[0]+ " ・"+str(pare[(feature[0], feature_ad[0])])+"回" + "  ・ " + \
                  str(int(100*((pare[(feature[0], feature_ad[0])]/(dic[feature[0]]+1))/((all_dinner_pare[(feature[0], feature_ad[0])]+1)/(all_review_word_dinner[feature[0]]+1))))/100)+"倍")
    l.append(ll)

In [17]:
l

[['赤酢 0回', '強い ・0回  ・ 0.0倍', 'あっさり ・0回  ・ 0.0倍', '酸っぱい ・0回  ・ 0.0倍'],
 ['握り 73回', '大きい ・1回  ・ 0.33倍', '小さい ・5回  ・ 2.87倍', '創作 ・0回  ・ 0.0倍'],
 ['シャリ 27回',
  '大きい ・0回  ・ 0.0倍',
  '小さい ・1回  ・ 1.12倍',
  'パラパラ ・1回  ・ 4.08倍',
  '塩気 ・1回  ・ 0.46倍',
  '甘い ・0回  ・ 0.0倍',
  '酢 ・1回  ・ 0.16倍'],
 ['まぐろ 222回',
  'うまい ・33回  ・ 0.97倍',
  '臭い ・0回  ・ 0.0倍',
  'あっさり ・0回  ・ 0.0倍',
  'こってり ・0回  ・ 0.0倍',
  'とろける ・2回  ・ 0.62倍',
  '酸味 ・4回  ・ 1.48倍',
  '熟成 ・8回  ・ 0.89倍'],
 ['ワイン 0回', '多い ・0回  ・ 0.0倍', '相性 ・0回  ・ 0.0倍']]

In [20]:
columns = ["軸とその登場回数","修飾語・その登場回数・他の店に比べて何倍か","修飾語・その登場回数・他の店に比べて何倍か",
           "修飾語・その登場回数・他の店に比べて何倍か","修飾語・その登場回数・他の店に比べて何倍か",
          "修飾語・その登場回数・他の店に比べて何倍か","修飾語・その登場回数・他の店に比べて何倍か",
          "修飾語・その登場回数・他の店に比べて何倍か",]

In [21]:
df1 = pd.DataFrame(l, columns=columns)

In [22]:
df1

,軸とその登場回数,修飾語・その登場回数・他の店に比べて何倍か,修飾語・その登場回数・他の店に比べて何倍か,修飾語・その登場回数・他の店に比べて何倍か,修飾語・その登場回数・他の店に比べて何倍か,修飾語・その登場回数・他の店に比べて何倍か,修飾語・その登場回数・他の店に比べて何倍か,修飾語・その登場回数・他の店に比べて何倍か
0,赤酢 0回,強い ・0回 ・ 0.0倍,あっさり ・0回 ・ 0.0倍,酸っぱい ・0回 ・ 0.0倍,None,None,None,None
1,握り 73回,大きい ・1回 ・ 0.33倍,小さい ・5回 ・ 2.87倍,創作 ・0回 ・ 0.0倍,None,None,None,None
2,シャリ 27回,大きい ・0回 ・ 0.0倍,小さい ・1回 ・ 1.12倍,パラパラ ・1回 ・ 4.08倍,塩気 ・1回 ・ 0.46倍,甘い ・0回 ・ 0.0倍,酢 ・1回 ・ 0.16倍,None
3,まぐろ 222回,うまい ・33回 ・ 0.97倍,臭い ・0回 ・ 0.0倍,あっさり ・0回 ・ 0.0倍,こってり ・0回 ・ 0.0倍,とろける ・2回 ・ 0.62倍,酸味 ・4回 ・ 1.48倍,熟成 ・8回 ・ 0.89倍
4,ワイン 0回,多い ・0回 ・ 0.0倍,相性 ・0回 ・ 0.0倍,None,None,None,None,None


In [ ]:
all_dinner_pare

In [ ]:
all_review_word_dinner

In [ ]:
df1.to_csv("kiyoda_dinner_teian.csv", index=False, encoding="SHIFT-JIS")

In [ ]:
dic